In [1]:
import tensorflow as tf
import numpy as np
import json

In [2]:
pairs = json.load(open('../data/fb-pairs.json'))

In [3]:
print pairs[0]

[[u'i have to get a filling\non the 13th'], u'Tbh it does help']


In [5]:
chars = 'abcdefghijklmnopqrstuvwxyz'
# chars += chars.upper() # lowercase everything
chars += ' 1234567890/:\'"_-()\n'
NONE = 0
START = 1
END = 2
chars = [NONE, START, END] + list(chars)


In [6]:
CONTEXT_LEN = 64
RESPONSE_LEN = 64